In [1]:
import math
import time

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.lines import Line2D
import numpy as np
from casadi import *
from scipy.interpolate import CubicSpline

import importlib
import solvers_and_functions_package.Single_Wind as model
importlib.reload(model)


<module 'solvers_and_functions_package.Single_Wind' from 'C:\\Users\\irmas\\Documents\\TUM\\CaseCourse\\CasADi\\Main\\New_2D_model\\solvers_and_functions_package\\Single_Wind.py'>

# Wind model

In [ ]:
model.plot_wind(k=1,s=1.5,model=1,smooth=True)

# Pesch comparison

In [ ]:
k=1
s=1

A_w = model.A_wm1
B_w = model.B_wm1

integrator_b = model.rk4_step_bolza
integrator_h = model.rk4_step

pec = True
tol = 1e-10
constr_viol_tol = 1e-6

wb,Jb = model.solver_bolza_scaled(k_value=k, s_value=s,A_w=A_w,B_w=B_w,pesch_end_cond=pec,integrator=integrator_b,N=320)
wh,Jh = model.solver_min_h_scaled(k_value=k, s_value=s,A_w=A_w,B_w=B_w,pesch_end_cond=pec,integrator=integrator_h,N=320)

sol_b=model.sol_dictonary(wb,True,True)
sol_h=model.sol_dictonary(wh,False,True)

print(f"min altitude, bolza: {min(sol_b['h'])}")
print(f"min altitude, min h: {min(sol_h['h'])}")

print(f"bolza obj, bolza: {Jb.item()*model.params.inv_scale_Q}")
print(f"bolza obj, min h: {Jh.item()*model.params.inv_scale_Q}")

# Prep
A = np.vectorize(A_w)
B = np.vectorize(B_w)

x_grid = np.linspace(0, 10000, 200)
h_grid = np.linspace(0, 1500, 100)
X, H = np.meshgrid(x_grid, h_grid)

U = k * A(X,s)
V = k * H * B(X,s) / model.params.h_star


In [ ]:
# Bolza

In [ ]:
A_w=model.A_wm1
B_w=model.B_wm1
integrator = model.rk2_step_bolza
pec = False
N = 320
k=1
s=1

sols=[]

q=6
scale_Q = 1/10**(17)
w, J = model.solver_bolza_scaled(k_value=k,s_value=s,A_w=A_w,B_w=B_w,N=N,integrator=integrator,pesch_end_cond=pec,q=q,scale_Q=scale_Q)
sol=model.sol_dictonary(w,True,True)
sols.append(sol)

q=8
scale_Q = 1/10**(22)
w, J = model.solver_bolza_scaled(k_value=k,s_value=s,A_w=A_w,B_w=B_w,N=N,integrator=integrator,pesch_end_cond=pec,q=q,scale_Q=scale_Q)
sol=model.sol_dictonary(w,True,True)
sols.append(sol)

q=20
scale_Q = 1/10**(55)
w, J = model.solver_bolza_scaled(k_value=k,s_value=s,A_w=A_w,B_w=B_w,N=N,integrator=integrator,pesch_end_cond=pec,q=q,scale_Q=scale_Q)
sol=model.sol_dictonary(w,True,True)
sols.append(sol)

q=50
scale_Q = 1/10**(135)
w, J = model.solver_bolza_scaled(k_value=k,s_value=s,A_w=A_w,B_w=B_w,N=N,integrator=integrator,pesch_end_cond=pec,q=q,scale_Q=scale_Q)
sol=model.sol_dictonary(w,True,True)
sols.append(sol)

q=100
scale_Q = 1/10**(270)
w, J = model.solver_bolza_scaled(k_value=k,s_value=s,A_w=A_w,B_w=B_w,N=N,integrator=integrator,pesch_end_cond=pec,q=q,scale_Q=scale_Q)
sol=model.sol_dictonary(w,True,True)
sols.append(sol)

w,J =model.solver_min_h_scaled(k_value=k,s_value=s,A_w=A_w,B_w=B_w,N=N,integrator=model.rk2_step,pesch_end_cond=pec)
sol=model.sol_dictonary(w,False,True)
sols.append(sol)

labels=['q=6','q=8','q=20','q=50','q=100','max min h']

plt.figure()
for i in range(len(sols)-1):
    plt.plot(sols[i]['x'],sols[i]['h'],'--',label=labels[i])
    print(min(sols[i]['h']))
plt.plot(sols[len(sols)-1]['x'],sols[len(sols)-1]['h'],label=labels[len(sols)-1])
print(min(sols[len(sols)-1]['h']))
plt.grid()
plt.legend()
plt.xlabel('horizontal distance [ft]')
plt.ylabel('altitude [ft]')
plt.show()

plt.figure()
for i in range(len(sols)-1):
    plt.plot(sols[i]['t_grid'][:-1],sols[i]['u'],'--',label=labels[i])
plt.plot(sols[len(sols)-1]['t_grid'][:-1],sols[len(sols)-1]['u'],label=labels[len(sols)-1])
plt.grid()
plt.legend()
plt.xlabel('horizontal distance [ft]')
plt.ylabel('altitude [ft]')



A_w=model.A_wm1
B_w=model.B_wm1
integrator = model.rk4_step
pec = False
N = 320

sols = []
k_list = [0.75, 1, 1.25]
M = len(k_list)

for i in range(M):
    k=k_list[i]
    s=1/k**2
    w, J = model.solver_min_h_scaled(k_value=k,s_value=s,A_w=A_w,B_w=B_w,N=N,integrator=integrator,pesch_end_cond=pec)
    sol = model.sol_dictonary(w,False,True)
    sols.append(sol)


for i in range(M):
    print(float(min(sols[i]['h'])))# Max min height
    

plt.figure(figsize=(12, 6))
for i in range(M):
    plt.plot(sols[i]['x'], sols[i]['h'], '--', label=rf"k={k_list[i]}, s=1/k^2")
plt.xlabel('horizontal distance [ft]')
plt.ylabel('altitude [ft]')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
for i in range(M):
    plt.step(sols[i]['t_grid'][:-1], sols[i]['u'], label=rf"k=1, s={s_list[i]}")
plt.xlabel('t [s]')
plt.legend()
plt.grid()
plt.show()

# MC

In [ ]:
# vary k and s
A_w=model.A_wm1
B_w=model.B_wm1
integrator = model.rk4_step
pec = False
N = 160

k=1
s=1
w, J = model.solver_min_h_scaled(k_value=k,s_value=s,A_w=A_w,B_w=B_w,N=N,integrator=integrator,pesch_end_cond=pec)
u = model.u_opt_return(w,False,True)

violating_ks = model.MC_simulations(u,10000,A_w,B_w,MC_type=2,k_mid=1,s_mid=1,std_k=0.25/3, integrator=model.MC_rk4_step)
print(violating_ks)

# Multi Wind

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from casadi import *

import importlib
import solvers_and_functions_package.Multi_wind as model
importlib.reload(model)



In [ ]:
A_w=model.A_wm1
B_w=model.B_wm1
integrator = model.rk4_step
pec = False
N = 160

sols = []
#k_list = [0.75, 1, 1.25]
#s_list = [1,1,1]

k_list = [0.75, 1, 1.25]
s_list = [1/0.75**2,1,1/1.25**2]

#k_list = [1,1,1]
#s_list = [0.75, 1, 1.25]
M = len(k_list)

w, J = model.solver_min_h_scaled(k_values=k_list,s_values=s_list,A_w=A_w,B_w=B_w,N=N,integrator=integrator)
model.ploter(w,k_list,s_list,N=N,is_scaled=True,is_bolza=False)


# MC

In [ ]:

A_w=model.A_wm1
B_w=model.B_wm1
integrator = model.rk4_step
N = 160
k = [0.75,1,1.25]
s = [1/0.75**2,1,1/1.25**2]
w, J = model.solver_min_h_scaled(k_values=k,s_values=s,A_w=A_w,B_w=B_w,N=N,integrator=integrator)
u = model.u_opt_return(w,N,True)

violating_ks = model.MC_simulations(u,10000,A_w,B_w,2,1,1,0.25/3, integrator=model.MC_rk4_step)
print(violating_ks)